In [1]:
from dataclasses import dataclass
import pandas as pd
from multicall import Call

from mainnet_launch.constants import (
    ALL_CHAINS,
    ROOT_PRICE_ORACLE,
    INCENTIVE_PRICING_STATS,
    LIQUIDATION_ROW,
    profile_function,
    ChainData,
    ETH_CHAIN,
)
from mainnet_launch.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    get_state_by_one_block,
    identity_with_bool_success,
    safe_normalize_with_bool_success,
)
from mainnet_launch.database.schema.postgres_operations import _exec_sql_and_cache
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
    insert_avoid_conflicts,
)
from mainnet_launch.database.schema.views import get_token_details_dict


from mainnet_launch.constants import *
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI
import pandas as pd

# What we care about for incentive tokens

- How accurate is our selling of incentive tokens acheived and oracle token prices
- How frequent is our selling of incentive tokens (for the base asset)
- How fequently are we claiming tokens,

    - be fine with false positives

In [ ]:
from mainnet_launch.database.schema.postgres_operations import get_subset_not_already_in_column, insert_avoid_conflicts
from mainnet_launch.database.schema.full import Tokens
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
)
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_tokens_and_autopoolDestinations_table import (
    ensure_all_tokens_are_saved_in_db,
)


def _get_highest_swapped_event_already_fetched():
    # stub
    highest_block_already_fetched = {}
    for liquidation_row in [LIQUIDATION_ROW, LIQUIDATION_ROW2]:
        for chain in ALL_CHAINS:
            highest_block_already_fetched[(chain, liquidation_row(chain))] = chain.block_autopool_first_deployed
    return highest_block_already_fetched


def _fetch_new_swapped_events() -> pd.DataFrame:
    all_swapped_events = []
    highest_block_already_fetched = _get_highest_swapped_event_already_fetched()

    for chain in ALL_CHAINS:
        token_addresses_to_ensure_we_have_in_db = set()
        for liquidation_row in [LIQUIDATION_ROW, LIQUIDATION_ROW2]:
            start_block = highest_block_already_fetched[(chain, liquidation_row(chain))] + 1
            contract = chain.client.eth.contract(liquidation_row(chain), abi=DESTINATION_DEBT_REPORTING_SWAPPED_ABI)
            swapped_df = fetch_events(contract.events.Swapped, chain=chain, start_block=start_block)
            swapped_df["liquidation_row"] = liquidation_row(chain)

            if not swapped_df.empty:
                all_swapped_events.append(swapped_df)

            token_addresses_to_ensure_we_have_in_db.update(set(swapped_df["sellTokenAddress"].unique()))
            token_addresses_to_ensure_we_have_in_db.update(set(swapped_df["buyTokenAddress"].unique()))

        ensure_all_tokens_are_saved_in_db(list(token_addresses_to_ensure_we_have_in_db), chain)

    if all_swapped_events:
        all_swapped_events = pd.concat(all_swapped_events)
        token_to_decimals, token_to_symbol = get_token_details_dict()
        all_swapped_events = _add_token_details(all_swapped_events, token_to_decimals, token_to_symbol)
    else:
        all_swapped_events = pd.DataFrame()
    return all_swapped_events


def _add_token_details(
    all_swapped_events: pd.DataFrame, token_to_decimals: dict, token_to_symbol: dict
) -> pd.DataFrame:
    all_swapped_events["sellTokenAddress"] = all_swapped_events["sellTokenAddress"].apply(
        lambda x: Web3.toChecksumAddress(x)
    )
    all_swapped_events["buyTokenAddress"] = all_swapped_events["buyTokenAddress"].apply(
        lambda x: Web3.toChecksumAddress(x)
    )
    all_swapped_events["sellTokenAddress_decimals"] = all_swapped_events["sellTokenAddress"].map(token_to_decimals)
    all_swapped_events["buyTokenAddress_decimals"] = all_swapped_events["buyTokenAddress"].map(token_to_decimals)
    all_swapped_events["sellTokenAddress_symbol"] = all_swapped_events["sellTokenAddress"].map(token_to_symbol)
    all_swapped_events["buyTokenAddress_symbol"] = all_swapped_events["buyTokenAddress"].map(token_to_symbol)

    all_swapped_events["sellAmount_normalized"] = all_swapped_events["sellAmount"] / (
        10 ** all_swapped_events["sellTokenAddress_decimals"]
    )
    all_swapped_events["buyAmount_normalized"] = all_swapped_events["buyAmount"] / (
        10 ** all_swapped_events["buyTokenAddress_decimals"]
    )
    all_swapped_events["buyTokenAmountReceived_normalized"] = all_swapped_events["buyTokenAmountReceived"] / (
        10 ** all_swapped_events["buyTokenAddress_decimals"]
    )

    return all_swapped_events


all_swapped_events = _fetch_new_swapped_events()
all_swapped_events

,event,block,transaction_index,log_index,hash,sellTokenAddress,buyTokenAddress,sellAmount,buyAmount,buyTokenAmountReceived,liquidation_row,sellTokenAddress_decimals,buyTokenAddress_decimals,sellTokenAddress_symbol,buyTokenAddress_symbol,sellAmount_normalized,buyAmount_normalized,buyTokenAmountReceived_normalized
0,Swapped,20786622,89,212,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,405885564600978683213,65660904437741059,65660904437741083,0xBf58810BB1946429830C1f12205331608c470ff5,NaN,18.0,NaN,WETH,NaN,0.065661,0.065661
1,Swapped,20786622,89,239,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xba100000625a3754423978a60c9317c58a424e3D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,97656862179630371841,73583617510509109,73583617510509114,0xBf58810BB1946429830C1f12205331608c470ff5,NaN,18.0,NaN,WETH,NaN,0.073584,0.073584
2,Swapped,20786622,89,263,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,305110013,123151123466434536,123151125084164991,0xBf58810BB1946429830C1f12205331608c470ff5,6.0,18.0,USDC,WETH,305.110013,0.123151,0.123151
3,Swapped,20786622,89,283,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,6322527432225350368,5347173311905731,5347173311905731,0xBf58810BB1946429830C1f12205331608c470ff5,NaN,18.0,NaN,WETH,NaN,0.005347,0.005347
4,Swapped,20786622,89,307,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xD533a949740bb3306d119CC777fa900bA034cd52,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2107509144075116791221,241327419724972605,241327419724972627,0xBf58810BB1946429830C1f12205331608c470ff5,NaN,18.0,NaN,WETH,NaN,0.241327,0.241327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,Swapped,34798196,274,449,0xf779d75566c629c255521199c80f00af9b2cb30d59d0...,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0x4200000000000000000000000000000000000006,38318377636359479460,1575648151383780,1586755439469758,0x7571dE594A92379c0A053E2A5004514057c10B5D,NaN,18.0,NaN,WETH,NaN,0.001576,0.001587
1136,Swapped,34798196,274,468,0xf779d75566c629c255521199c80f00af9b2cb30d59d0...,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1,0x4200000000000000000000000000000000000006,17842665391956125838,5078846050078530,5114648590214063,0x7571dE594A92379c0A053E2A5004514057c10B5D,NaN,18.0,NaN,WETH,NaN,0.005079,0.005115
1137,Swapped,34798209,15,21,0xc1572d07a99e09b595b061552ca3c6bdfdd8f2694df4...,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,111543692876251106166,21109750,21258726,0x7571dE594A92379c0A053E2A5004514057c10B5D,NaN,6.0,NaN,USDC,NaN,21.10975,21.258726
1138,Swapped,34798209,15,34,0xc1572d07a99e09b595b061552ca3c6bdfdd8f2694df4...,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,146754438351657322006,145663170,146692499,0x7571dE594A92379c0A053E2A5004514057c10B5D,18.0,6.0,GHO,USDC,146.754438,145.66317,146.692499


In [ ]:
# add new tokens to tokens table if needed

In [ ]:
@dataclass
class Swapped:
    tx_hash: str  # primary keys
    log_index: int  # primary keys
    sell_token_address: str
    buy_token_address: str
    sell_amount: float
    buy_amount: float
    buyTokenAddress_amount_received: float
    liquidation_row_contract_address: str


@dataclass
class VaultLiquidated:
    """Reward tokens are sold for base asset. This at a per destiantion vault level"""

    tx_hash: str  # primary keys
    log_index: int  # primary keys

    destination_vault_address: str
    from_token_address: str
    to_token_address: str

    liquidated_amount: float  # in terms of to_token_address
    liquidation_row_contract_address: str


# https://sonicscan.org/tx/0x063b1707c4ad86daa03568bd1758dcbac73c4038d127e0342a1b11d555964622
# this tells me when a vault was liqudiated, (what and how much)
# we can tell what we price we got and when.

In [6]:
# oracle_price_calls = [
#     Call(
#         INCENTIVE_PRICING_STATS(chain),
#         ["getPrice(address,uint40)((uint256,uint256))", addr, 2 * 86400],  # 2 days of latency
#         [(addr, _min_of_low_and_high_price)],
#     )
#     for addr in token_addresses
# ]


# emit Swapped(address(sellToken), address(buyToken), sellAmount, buyTokenAmountReceived, buyTokenAmountReceived);

# interface IAsyncSwapper {
#     error TokenAddressZero();
#     error SwapFailed();
#     error InsufficientBuyAmountReceived(uint256 buyTokenAmountReceived, uint256 buyAmount);
#     error InsufficientSellAmount();
#     error InsufficientBuyAmount();
#     error InsufficientBalance(uint256 balanceNeeded, uint256 balanceAvailable);

#     event Swapped(
#         address indexed sellTokenAddress,
#         address indexed buyTokenAddress,
#         uint256 sellAmount,
#         uint256 buyAmount,
#         uint256 buyTokenAmountReceived
#     );

#     /**
#      * @notice Swaps sellToken for buyToken
#      * @param swapParams Encoded swap data
#      * @return buyTokenAmountReceived The amount of buyToken received from the swap
#      */
#     function swap(
#         SwapParams memory swapParams
#     ) external payable returns (uint256 buyTokenAmountReceived);
# }


# swapped event

# event Swapped(
#     address indexed sellTokenAddress,
#     address indexed buyTokenAddress,
#     uint256 sellAmount,
#     uint256 buyAmount,
#     uint256 buyTokenAmountReceived
# );


# https://etherscan.io/tx/0x052b4231be3c2b28480b335085cad1c20ba838cccd5fc98e9c1d39e8502c9f11#eventlog

# VaultLiquidated (index_topic_1 address vault, index_topic_2 address fromToken, index_topic_3 address toToken, uint256 amount)View Source

# 133

# Topics
# 0 0x0272b5a6ff5cab190795f808ef35307240b8bc0011849cb8e15c093b40b22dfb
# 1: vault
# 0x0091Fec1B75013D1b83f4Bb82f0BEC4E256758CB # Tokemak-Dola USD Stablecoin-DOLA/sUSDe string
# 2: fromToken
# 0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B # CVX
# 3: toToken
# 0x865377367054516e17014CcdED1e7d814EDC9ce4 # dola
# Data


# amount :
# 583413391926390257

# at a vault level?


# https://etherscan.io/address/0xe2c7011866db4cc754f1b9b60b2f2999b5b54be4#code


# we also want reward added events